<h1 style = "text-align: center"> <b>Method 1: Color-based Technique</b></h1> 

## **1. Libraries**

In [1]:
import matplotlib.pyplot as plt
import cv2
import numpy as np

## **2. Read video**

In [24]:
# read video
video_path = "/content/classroom_video.mp4"
video = cv2.VideoCapture(video_path)
video_frames = []

if not video.isOpened():
  print("Cannot open the video")

while True:
  ret, frame = video.read()
  if not ret:
    break
  video_frames.append(frame)
video.release()

print(f"Total frames: {len(video_frames)}")



Total frames: 256


## **3. Color-based technique**

In [ ]:
def extractByPixel(frames):
  def compute_diff_frames(frame1, frame2):
    # convert to HSV
    frame1_hsv = cv2.cvtColor(frame1, cv2.COLOR_BGR2HSV)
    frame2_hsv = cv2.cvtColor(frame2, cv2.COLOR_BGR2HSV)

    # compute difference
    diff = np.abs(frame1_hsv - frame2_hsv)

    return np.sum(diff)

  def isGreater(a, b):
    return a>b

  def boolrelextrema(data,
                      comparator,
                      axis=0,
                      order=1,
                      mode='clip'):

    if((not isinstance(order, int)) or (order < 1)):
      raise ValueError('Order must be an int >= 1')

    # get length of the array
    data = np.array(data)
    data_len = data.shape[axis]
    locs = np.arange(0, data_len)

    # create an array with all values set to True and its size = data_len
    results = np.ones(data_len, dtype=bool)
    main = data.take(locs, axis = axis, mode = mode)

    # Iterate over each shift from 1 to order (inclusive)
    for shift in range(1, order+1):
      plus = data.take(locs + shift, axis = axis, mode = mode)
      minus = data.take(locs - shift, axis = axis, mode = mode)

      results &= comparator(main, plus)
      results &= comparator(main, minus)

      if ~results.any():
        return results

    return results

  def argrelextrema(data,
                    comparator,
                    axis=0,
                    order=1,
                    mode='clip'):


    results = boolrelextrema(data, comparator, axis, order, mode)
    # print(results)
    return np.where(results)[0]

  import numpy as np

  def smooth(x, len_slide_window, type_window="hanning"):
      x = np.array(x)
      if x.ndim != 1:
          raise ValueError("smooth only accepts 1 dimension arrays.")
      if x.size < len_slide_window:
          raise ValueError("Input vector needs to be bigger than window size.")
      if len_slide_window < 3:
          return x

      if type_window not in ["flat", "hanning", "hamming", "bartlett", "blackman"]:
          raise ValueError(
              "Smoothing Window must be one of 'flat', 'hanning', 'hamming', 'bartlett', 'blackman'"
          )

      # Extend the signal at the boundaries
      s = np.r_[
          2 * x[0] - x[len_slide_window:1:-1],
          x,
          2 * x[-1] - x[-1:-len_slide_window:-1],
      ]

      # Define the window
      if type_window == "flat":
          w = np.ones(len_slide_window, "d")
      else:
          w = getattr(np, type_window)(len_slide_window)

      # Convolve the signal with the window
      y = np.convolve(w / w.sum(), s, mode="same")
      return y[len_slide_window - 1 : -len_slide_window + 1]

  diff_frames = []
  for i in range(1, len(frames)) :
    diff_frames.append(compute_diff_frames(frames[i-1], frames[i]))

  smoothed_diff = smooth(diff_frames, 10, 'hanning')

  return argrelextrema(smoothed_diff, isGreater)


(25,
 array([  7,  14,  22,  30,  39,  60,  70,  75,  81,  94, 100, 105, 110,
        121, 141, 150, 157, 163, 180, 186, 190, 217, 227, 237, 243]))

## **4. Test technique**

In [ ]:
keyframes_byPixel = extractByPixel(video_frames)
len(keyframes_byPixel), keyframes_byPixel